In [36]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
import hashlib

In [37]:
data = pd.read_csv('results.csv')
data = data.fillna({"TARGET_ACCOUNT": "UNKNOWN"})
data = data.fillna({"GDDB_ROLE_SPECIAL": "STANDARD"})
_CSV_COLUMNS = ['USERNAME','EVENT_ID','TARGET_ACCOUNT','TARGET_UNIQUE_ID','TARGET_GENDER','GDDB_ROLE','GDDB_ROLE_SPECIAL','LABEL']
data.head()

,USERNAME,EVENT_ID,TARGET_ACCOUNT,TARGET_UNIQUE_ID,TARGET_GENDER,GDDB_ROLE,GDDB_ROLE_SPECIAL,LABEL
0,TROMMRE1,GDDB04,EUNET/BOHNSRO1,BOHNSRO1,Person,ROLE_GDDB_ADMINISTRATOR - none - HELPDESK,STANDARD,incident
1,TROMMRE1,GDDB04,EUNET/KOENIKO1,KOENIKO1,Person,ROLE_GDDB_ADMINISTRATOR - none - HELPDESK,STANDARD,incident
2,COLLAAR1,GDDB04,NANET/CUARTJO1,CUARTJO1,Person,ROLE_GDDB_ADMINISTRATOR - none - HELPDESK,STANDARD,incident
3,DUEBLDO1,GDDB02,EUNET/RELEA201,RELEA201,Person,ROLE_GDDB_MANAGER,STANDARD,exeption
4,DUEBLDO1,GDDB02,EUNET/ADM_RELEA201,RELEA201,Person,ROLE_GDDB_MANAGER,STANDARD,exeption


In [38]:
features = dict(zip(_CSV_COLUMNS, data))

In [39]:
features

{'EVENT_ID': 'EVENT_ID',
 'GDDB_ROLE': 'GDDB_ROLE',
 'GDDB_ROLE_SPECIAL': 'GDDB_ROLE_SPECIAL',
 'LABEL': 'LABEL',
 'TARGET_ACCOUNT': 'TARGET_ACCOUNT',
 'TARGET_GENDER': 'TARGET_GENDER',
 'TARGET_UNIQUE_ID': 'TARGET_UNIQUE_ID',
 'USERNAME': 'USERNAME'}

In [40]:
#data['USERNAME'] = data['USERNAME'].apply(lambda x: int(hashlib.md5(x.encode()).hexdigest()[:8], 16))
#data["EVENT_ID"] = le.fit_transform(data['EVENT_ID'])
#data['TARGET_ACCOUNT'] = data['TARGET_ACCOUNT'].apply(lambda x: int(hashlib.md5(x.encode()).hexdigest()[:8], 16))
#data['TARGET_UNIQUE_ID'] = data['TARGET_UNIQUE_ID'].apply(lambda x: int(hashlib.md5(x.encode()).hexdigest()[:8], 16))
#data["TARGET_GENDER"] = le.fit_transform(data['TARGET_GENDER'])
#data["GDDB_ROLE"] = le.fit_transform(data['GDDB_ROLE'])
#data["GDDB_ROLE_SPECIAL"] = le.fit_transform(data['GDDB_ROLE_SPECIAL'])
#data["LABEL"] = le.fit_transform(data['LABEL'])

In [41]:
uid = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('USERNAME', hash_bucket_size=10000),1)
eid = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_vocabulary_list('EVENT_ID',['GDDB01','GDDB02','GDDB03','GDDB04']),1)
ta = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('TARGET_ACCOUNT', hash_bucket_size=10000),1)
tuid = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('TARGET_UNIQUE_ID', hash_bucket_size=10000),1)
tg = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_vocabulary_list('TARGET_GENDER',['Person','Generic']),1)
role = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('GDDB_ROLE', hash_bucket_size=10),1)
srole = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_vocabulary_list('GDDB_ROLE_SPECIAL',['ROLE_GDDB_AD','ROLE_GDDB_LEGAL']),1)

In [42]:
feat_cols = [uid,eid,ta,tuid,tg,role,srole]

In [43]:
x_data = data.drop('LABEL',axis=1)

In [44]:
labels = data['LABEL']

In [45]:
#x_data

In [46]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.3, random_state=101)

In [47]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=12,num_epochs=1000,shuffle=True)

In [48]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020021086550>, '_log_step_count_steps': 100, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_service': None, '_tf_random_seed': None, '_task_id': 0, '_master': '', '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_session_config': None, '_model_dir': 'C:\\Users\\Gabor\\AppData\\Local\\Temp\\tmpahf5y46i', '_num_worker_replicas': 1}


In [49]:
dnn_model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.


UnimplementedError: Cast string to float is not supported
	 [[Node: dnn/head/ToFloat = Cast[DstT=DT_FLOAT, SrcT=DT_STRING, _device="/job:localhost/replica:0/task:0/device:CPU:0"](dnn/head/labels)]]

Caused by op 'dnn/head/ToFloat', defined at:
  File "C:\Users\Gabor\Anaconda3\envs\tfdeeplearning\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Gabor\Anaconda3\envs\tfdeeplearning\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Gabor\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Gabor\Anaconda3\envs\tfdeeplearning\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Gabor\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Gabor\Anaconda3\envs\tfdeeplearning\lib\site-p

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)

In [175]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2017-12-17-21:39:22
INFO:tensorflow:Restoring parameters from C:\Users\Gabor\AppData\Local\Temp\tmpdpl5mxo1\model.ckpt-10000
INFO:tensorflow:Finished evaluation at 2017-12-17-21:39:23
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.820093, accuracy_baseline = 0.820093, auc = 0.5, auc_precision_recall = 0.589953, average_loss = 0.477172, global_step = 10000, label/mean = 0.179907, loss = 4.74952, prediction/mean = 0.140714


{'accuracy': 0.82009345,
 'accuracy_baseline': 0.82009345,
 'auc': 0.5,
 'auc_precision_recall': 0.5899533,
 'average_loss': 0.47717157,
 'global_step': 10000,
 'label/mean': 0.17990655,
 'loss': 4.7495217,
 'prediction/mean': 0.14071353}